In [12]:
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras.utils import to_categorical
from itertools import permutations 

resultados = np.zeros((3912,10))

def save_model(network):
    
    model_json = network.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    network.save_weights("model.h5")

def load_model():
    
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = models.model_from_json(loaded_model_json)
    loaded_model.load_weights("model.h5")
    return loaded_model

def combine(units,number_of_layers):
    
    mixed_vals = [item for t in list(permutations(units,number_of_layers)) for item in t]
    return mixed_vals

def maximum_model_efficiency(history_dict):
    
    validation_accuracy_values = np.zeros(repeticiones)
    c=0
    for i in history_dict['val_accuracy']:
        
        if c < (repeticiones):
            
            validation_accuracy_values[c] = i
            c+=1
            
    val_acc_max = np.amax(validation_accuracy_values)
    index = np.where(validation_accuracy_values == val_acc_max)
    maximum_eff = index[0] + 1
    
def net_build(units,number_of_layers,activation_func,iterations):
    
    row = 0
    
    for h in range (len(activation_func)):
        
        for i in range (len(number_of_layers)):

            a = combine(units,number_of_layers[i])

            cardboard_f = pd.read_excel("cardboard.xlsx", header=None)
            cardboard_f = cardboard_f.to_numpy()
            np.random.shuffle(cardboard_f)

            cardboard_f_train = cardboard_f[:900,:10]
            cardboard_f_test = cardboard_f[900:,:10]

            train_labels = cardboard_f[:900,10]
            test_labels = cardboard_f[900:,10]

            len(train_labels)
            train_labels = train_labels.astype(int)

            len(test_labels)
            test_labels = test_labels.astype(int)

            train_labels = to_categorical(train_labels)
            network = models.Sequential()

            var_ns = ''

            for j in range (len(a)):

                network.add(layers.Dense(a[j], activation=activation_func[h], input_shape=(10,)))

                var_ns = var_ns + str(a[j])

                if((j+1)%number_of_layers[i] == 0):

                    network.add(layers.Dense(2, activation='softmax'))
                    network.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

                    x_val = cardboard_f_train[:450]
                    partial_x_train = cardboard_f_train[450:]

                    y_val = train_labels[:450]
                    partial_y_train = train_labels[450:]

                    history = network.fit(partial_x_train,
                                        partial_y_train,
                                        epochs=repeticiones,
                                        batch_size=128,
                                        validation_data=(x_val, y_val),
                                        verbose=0)

                    history_dict = history.history
                    prediction_vector=network.predict(cardboard_f_test)

                    type_zero_count = 0
                    type_one_count = 0
                    guesses = np.ones(len(prediction_vector))

                    for j in range (len(prediction_vector)):
                        if prediction_vector[j,0] > prediction_vector[j,1]:
                            type_zero_count += 1
                            guesses[j] = 0
                        else:
                            type_one_count += 1
                            guesses[j] = 1

                    hits = 0
                    for k in range (len(guesses)):    
                        if guesses[k] == test_labels[k]:
                            hits += 1

                    non_hits = len(guesses) - hits
                    efficiency = float("{:0.2f}".format(hits/len(guesses)*100))

                    validation_accuracy_values=np.zeros(repeticiones)
                    c=0
                    for m in history_dict['val_accuracy']:

                        if c < (repeticiones):

                            validation_accuracy_values[c] = m
                            c+=1

                    val_acc_max = np.amax(validation_accuracy_values)
                    index = np.where(validation_accuracy_values == val_acc_max)[0].tolist()[0]

                    resultados[row,0] = type_zero_count
                    resultados[row,1] = type_one_count
                    resultados[row,2] = hits
                    resultados[row,3] = non_hits
                    resultados[row,4] = efficiency
                    resultados[row,5] = float("{:0.2f}".format(np.amax(validation_accuracy_values)*100))
                    resultados[row,6] = index
                    resultados[row,7] = number_of_layers[i]
                    resultados[row,8] = int(var_ns)
                    resultados[row,9] = h + 1
                    var_ns = ''
                    row += 1
                    
                    if (efficiency > 92.7):
                        save_model(network)

In [13]:
unidades_vals = [30,60,90,110]
num_capas=[1,2,3]
f_activacion = ['relu']
repeticiones=750

net_build(unidades_vals,num_capas,f_activacion,repeticiones)

In [14]:
resultados_dataFrame=pd.DataFrame(resultados,columns=['Type A cardboard','Type B cardboard','Hits'
                                                      ,'Non-hits','Test Accuracy (%)','Max validation accuracy (%)'
                                                      ,'Max accuracy epoch number','Number of layers'
                                                      ,'Number of neurons'
                                                      ,'Activation function'])

resultados_dataFrame.nlargest(20,['Test Accuracy (%)'])

,Type A cardboard,Type B cardboard,Hits,Non-hits,Test Accuracy (%),Max validation accuracy (%),Max accuracy epoch number,Number of layers,Number of neurons,Activation function
10,485.0,415.0,840.0,60.0,93.33,92.89,65.0,2.0,9030.0,1.0
7,484.0,416.0,837.0,63.0,93.00,92.44,515.0,2.0,6030.0,1.0
11,454.0,446.0,831.0,69.0,92.33,93.11,311.0,2.0,9060.0,1.0
6,513.0,387.0,830.0,70.0,92.22,91.78,542.0,2.0,30110.0,1.0
9,443.0,457.0,830.0,70.0,92.22,92.89,339.0,2.0,60110.0,1.0
20,458.0,442.0,829.0,71.0,92.11,94.44,444.0,3.0,3011060.0,1.0
17,469.0,431.0,826.0,74.0,91.78,94.00,603.0,3.0,3060110.0,1.0
8,449.0,451.0,824.0,76.0,91.56,92.67,558.0,2.0,6090.0,1.0
21,430.0,470.0,819.0,81.0,91.00,94.44,514.0,3.0,3011090.0,1.0
19,439.0,461.0,814.0,86.0,90.44,94.22,163.0,3.0,3090110.0,1.0
